In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client


In [3]:
target = new_client.target
target_query = target.search('coronavirus')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Coronavirus,Coronavirus,17.0,False,CHEMBL613732,[],ORGANISM,11119
1,[],SARS coronavirus,SARS coronavirus,15.0,False,CHEMBL612575,[],ORGANISM,227859
2,[],Feline coronavirus,Feline coronavirus,15.0,False,CHEMBL612744,[],ORGANISM,12663
3,[],Human coronavirus 229E,Human coronavirus 229E,13.0,False,CHEMBL613837,[],ORGANISM,11137
4,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
5,[],Middle East respiratory syndrome-related coron...,Middle East respiratory syndrome-related coron...,9.0,False,CHEMBL4296578,[],ORGANISM,1335626
6,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.0,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
7,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


In [42]:
selected_target = targets.target_chembl_id[4]
selected_target
activity = new_client.activity

res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
df = pd.DataFrame.from_dict(res)

df.to_csv('acetylcholinesterase_01_bioactivity_data_raw.csv', index=False)

df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]

df2_nr = df2.drop_duplicates(['canonical_smiles'])

selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]

df3.to_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)

df4 = pd.read_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")


In [43]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL187579,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,7200.0,intermediate
1,CHEMBL188487,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,9400.0,intermediate
2,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500.0,inactive
3,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110.0,inactive
4,CHEMBL187717,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],2000.0,intermediate
...,...,...,...,...
81,CHEMBL2146517,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,21100.0,inactive
82,CHEMBL187460,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,226700.0,inactive
83,CHEMBL363535,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,38700.0,inactive
84,CHEMBL227075,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,14400.0,inactive


In [16]:
df5.to_csv('acetylcholinesterase_03_bioactivity_data_curated.csv', index=False)

In [17]:
! zip acetylcholinesterase.zip *.csv

  adding: acetylcholinesterase_01_bioactivity_data_raw.csv (deflated 90%)
  adding: acetylcholinesterase_02_bioactivity_data_preprocessed.csv (deflated 70%)
  adding: acetylcholinesterase_03_bioactivity_data_curated.csv (deflated 73%)
  adding: acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv (deflated 76%)
  adding: acetylcholinesterase_05_bioactivity_data_2class_pIC50.csv (deflated 76%)
  adding: acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv (deflated 96%)
  adding: acetylcholinesterase_07_bioactivity_data_2class_pIC50_pubchem_fp.csv (deflated 96%)


In [18]:
! ls -l

total 17404
-rw-rw-r-- 1 hschoung0124 hschoung0124   70334  9월 10 16:51 acetylcholinesterase_01_bioactivity_data_raw.csv
-rw-rw-r-- 1 hschoung0124 hschoung0124    6106  9월 10 16:52 acetylcholinesterase_02_bioactivity_data_preprocessed.csv
-rw-rw-r-- 1 hschoung0124 hschoung0124    6892  9월 10 16:52 acetylcholinesterase_03_bioactivity_data_curated.csv
-rw-rw-r-- 1 hschoung0124 hschoung0124  655414  9월 10 14:34 acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv
-rw-rw-r-- 1 hschoung0124 hschoung0124  489152  9월 10 14:34 acetylcholinesterase_05_bioactivity_data_2class_pIC50.csv
-rw-rw-r-- 1 hschoung0124 hschoung0124 8363909  9월 10 14:34 acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv
-rw-rw-r-- 1 hschoung0124 hschoung0124 6323895  9월 10 14:34 acetylcholinesterase_07_bioactivity_data_2class_pIC50_pubchem_fp.csv
-rw-rw-r-- 1 hschoung0124 hschoung0124  882090  9월 10 16:52 acetylcholinesterase.zip
-rw-rw-r-- 1 hschoung0124 hschoung0124  124747  9월 10 14:34 CDD_ML_Pa

In [4]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')


--2021-09-14 14:00:23--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.12’

Miniconda3-py37_4.8 100%[===================>]  81.12M   508KB/s    in 2m 53s  

2021-09-14 14:03:17 (480 KB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.12’ saved [85055499/85055499]

PREFIX=/usr/local
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: line 392: /usr/local/conda.exe: Permission denied
chmod: cannot access '/usr/local/conda.exe': No such file or directory
Unpacking payload ...
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: line 404: /usr/local/conda.exe: No such file or directory
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: line 406: /usr/local/conda.exe: No such fi